## Генераторы в обратную сторону
- метод `.send` на генераторе
- `yield` в обратную сторону
- `.throw`, `.close`

In [1]:
def powers(nums):
    for i in nums:
        yield i ** i

In [2]:
for i in powers(range(1, 6)):
    print(i)

1
4
27
256
3125


In [3]:
def waiter():
    print("what would you like")
    order = yield
    print("your order:", order)

In [4]:
w = waiter()
print(w)

<generator object waiter at 0x10445f100>


In [5]:
for i in waiter():
    print(i)

what would you like
None
your order: None


In [6]:
w = waiter()
w.send("pasta")

TypeError: can't send non-None value to a just-started generator

In [7]:
w = waiter()
next(w)

what would you like


In [8]:
w.send("pasta")

your order: pasta


StopIteration: 

In [9]:
w = waiter()
for _ in w:
    print("gonna order salad")
    w.send("fruit salad")
    print("that's it, thank you")

what would you like
gonna order salad
your order: fruit salad


StopIteration: 

In [10]:
w = waiter()
for _ in w:
    print("gonna order salad")
    w.send("fruit salad")
    print("that's it, thank you")
    break

what would you like
gonna order salad
your order: fruit salad


StopIteration: 

In [11]:
def waiter():
    order_items = []
    print("what would you like")
    while True:
        order = yield order_items
        order_items.append(order)
        print(order, "ok, what else")

In [12]:
w = waiter()
i_want_to_order = [
    "pasta",
    "juice",
    "fruit salad",
]
next(w)
for food in i_want_to_order:
    w.send(food)

what would you like
pasta ok, what else
juice ok, what else
fruit salad ok, what else


In [13]:
next(w)

None ok, what else


['pasta', 'juice', 'fruit salad', None]

In [14]:
def waiter():
    order_items = []
    print("what would you like")
    while True:
        order = yield order_items
        if order is None:
            print("done", order_items)
            return order_items
        order_items.append(order)
        print(order, "ok, what else")

In [15]:
w = waiter()
i_want_to_order = [
    "pasta",
    "juice",
    "fruit salad",
]
next(w)
for food in i_want_to_order:
    w.send(food)

what would you like
pasta ok, what else
juice ok, what else
fruit salad ok, what else


In [16]:
next(w)

done ['pasta', 'juice', 'fruit salad']


StopIteration: ['pasta', 'juice', 'fruit salad']

In [18]:
stop = object()

def waiter():
    order_items = []
    print("what would you like")
    while True:
        order = yield order_items
        if order is stop:
            print("done", order_items)
            return order_items
        if order is not None:
            order_items.append(order)
            print(order, "ok, what else")

In [19]:
w = waiter()
i_want_to_order = [
    "pasta",
    "juice",
    "fruit salad",
]

for food, already_ordered in zip(i_want_to_order, w):
    print("already_order", already_ordered)
    w.send(food)

what would you like
already_order []
pasta ok, what else
already_order ['pasta']
juice ok, what else
already_order ['pasta', 'juice']
fruit salad ok, what else


In [20]:
next(w)

['pasta', 'juice', 'fruit salad']

In [21]:
next(w)

['pasta', 'juice', 'fruit salad']

In [22]:
w.send(stop)

done ['pasta', 'juice', 'fruit salad']


StopIteration: ['pasta', 'juice', 'fruit salad']

In [23]:
class OrderComplete(Exception):
    pass


def waiter():
    order_items = []
    print("what would you like")
    while True:
        try:
            order = yield order_items
        except OrderComplete:
            print("done", order_items)
            return order_items
        if order is not None:
            order_items.append(order)
            print(order, "ok, what else")

In [24]:
w = waiter()
i_want_to_order = [
    "pasta",
    "juice",
    "fruit salad",
]

for food, already_ordered in zip(i_want_to_order, w):
    print("already_order", already_ordered)
    w.send(food)

what would you like
already_order []
pasta ok, what else
already_order ['pasta']
juice ok, what else
already_order ['pasta', 'juice']
fruit salad ok, what else


In [25]:
next(w)

['pasta', 'juice', 'fruit salad']

In [26]:
w.throw(OrderComplete)

done ['pasta', 'juice', 'fruit salad']


StopIteration: ['pasta', 'juice', 'fruit salad']

In [27]:
def powers(num):
    p = 0
    while True:
        p = yield num ** p

In [28]:
p = powers(2)
p.send(2)

TypeError: can't send non-None value to a just-started generator

In [29]:
next(p)

1

In [30]:
p.send(2)

4

In [31]:
p.send(3)

8

In [32]:
for i in range(5):
    print(i, p.send(i))

0 1
1 2
2 4
3 8
4 16


In [33]:
p.send(10)

1024

In [34]:
p.close()

In [35]:
p.send(10)

StopIteration: 

In [36]:
def items_by_one(seq):
    yield from seq

In [37]:
for i in items_by_one(range(3)):
    print(i)

0
1
2


In [40]:
def add_count():
    counter = 0
    while True:
        num = yield
        if num is None:
            return counter
        counter += num
        print("counter inc by", num, "now =", counter)


def collect_counts(counters: list):
    while True:
        counter = yield from add_count()
        counters.append(counter)

In [41]:
numbers = []
collector = collect_counts(numbers)
print(numbers)

[]


In [42]:
next(collector)

In [43]:
collector.send(5)

counter inc by 5 now = 5


In [44]:
collector.send(4)

counter inc by 4 now = 9


In [45]:
print(numbers)

[]


In [46]:
collector.send(None)

In [47]:
print(numbers)

[9]


In [48]:
collector.send(7)

counter inc by 7 now = 7


In [49]:
collector.send(3)

counter inc by 3 now = 10


In [50]:
print(numbers)
collector.send(None)
print(numbers)

[9]
[9, 10]
